### Original Codes

In [1]:
def left(i):
    return 2*(i) + 1 #as index starts from 0, actually the left will be the odd number

def right(i):
    return 2*(i+1) #as the index starts from 0, the actual right will be even

def heapify1(heap, i, t):
    """
    Parameters
    ----------
    heap : list of floats
        Assume that the heap size is the length of the heap
        
    Note
    ----
    No output is needed. This function should modify (if necessary) 
    heap in-place.
    
    """
    #i starts from 0
    heap_size = len(heap)-1 #as we assumed that the heap size = length of the heap above, but index number cannot go over heapsize-1 as index starts from 0
    L = left(i) 
    R = right(i)
    
    if L <= heap_size and heap[L][t]>heap[i][t]:
        largest = L
    else:
        largest = i
        
    if R <= heap_size and heap[R][t] > heap[largest][t]:
        largest = R
        
    if largest != i:
        heap[i], heap[largest] = heap[largest], heap[i] #exchange the numbers
        heapify1(heap, largest, t)

def build_max_heap_list(A,t):
    """
    Parameters
    ----------
    A : a list of floats
    
    Note
    ----
    No output is needed. The function should turn A into 
    a valid max heap, in-place.
    
    After building a max heap, do a heap sort
    
    """
    heap_size = len(A)-1
    for i in range(heap_size//2,-1,-1): #range from 0 to (floor of heap_size/2)-1
        heapify1(A,i,t)
        
def heap_sort(A,t):
    build_max_heap_list(A,t)
    for i in range(len(A)):
        B = A[i:]
        build_max_heap_list(B,1)
        A[i:] = B
    return A

from itertools import chain

### New Codes

In [2]:
class todo(object):    
    """
    This class builds each to do works independently.
    
    Attributes
    ---------
    ids : int
        It represents the id of that to-do. Assigned accordingly to input sequence, which means id does not matter that much
    
    description : str
        representing what to do
    
    duration : int
        representing the estimated duration in min.
    
    dependence: int
        representing what needs to be done before execution (if no dependencies, set to 0)
        int as an input, meaning that I want to do this after works after the 'dependence' is done represented by ids
    
    multi_tasking : int
        representing if it is multitasking possible work(1) or not(0)
        int as an input
        
    urgency : int
        d-day left for this thing to be done
        if d-day does not exist, enter -1
        
    importance : int
        0 : not important and trivial
        1 : not important in daily lives, but important in the far future (more than 5 years)
        2 : some what important in daily lives, as it will become important in near future (after 6 months ~ 4 years)
        3 : important to do in daily life, and it still will be in the future
        4 : whatever it is, it is very important for it to be done by the d-day(urgency)
        
    value : calculated from urgency and importance
        calculates the value of the work/do-do to be done, higher scores will be pre-assigned
        As a task with smaller d-day is more urgent thing to be done, the function goes like this.
        value = importance*10/(urgency+1) (if urgency >= 0)
        if it is not urgent, it will be equivalent to the urgency(d-day) set to 30
        
    time : float
        from 0 to 24, it represents the hour when the work is being done.
        automatically set to 0 in the first place before sorting
        this is here to be used by class 'make_a_todo_list'
    -------------
    self.task: list
        a list that holds all objects made from this class
    """
    
    def __init__(self, ids = 0, description = '', duration ='', dependence = None, multi_tasking =0, urgency = 0, importance = 0, time = 0):
        self.ids = ids #intobject
        self.description = description #str object
        self.duration = duration #float object, in min(ex, 1hr 30 min = 90)
        self.dependence = dependence #task's ids to be done before the activity
        self.multi_tasking = multi_tasking #int = multitasking possible with that work id, empty list= multitasking impossible
        if urgency >= 0 : #how important is this work? Is it urgent? Is it important but not urgent?
            self.value = (importance*10)/(urgency+1)
        else: #base rate of the importance will be d-30
            self.value = (1/31)*(importance*10) 
        self.time = time #means that it is not yet set when to do the work, time refers to the hour when to do the work(ex, time = 13 means the work will start from 1pm)
        
    def aslist(self):
        #return the todo list
        return [self.ids, self.description, self.duration, self.dependence, self.multi_tasking, self.value, self.time, 0]
        
    
class todo_list(object): #make a list of given classes
    """
    After creating objects from class todo, create the list what task should I do to make my day efficient
    tasks is the list of objects created by class todo
    wakeup_hr is there to set the time for waking up. This can be changed by wakeup_change method
    """
    
    def __init__(self, tasks, wakeup_hr = 8):
        self.tasks = tasks
        #wake up has a value of 100, because if we don't wake up, there will be no start of the day
        self.wake_up = [0, 'wakeup', 0, [], 0, 100, wakeup_hr] 
        self.hr = [[self.wake_up[6], self.wake_up[6], self.wake_up]]
        
    def wakeup_change(self):
        wakeup_time = int(input("When do you want to start your day tomorrow?"))
        self.wake_up = [1,'wakeup', 0, [], 0, 100, wakeup_time]
        self.hr =  [[self.wake_up[6], self.wake_up[6], self.wake_up]]
        
    def cluster(self):
        #before sorting, we need to make a cluster of tasks by dependency and dependencies from object made from class todo.
        #if the dependency is true, it means that I will need to finish this right after the work 'dependencies' are done
        clustered_tasks = []
        for i in range(len(self.tasks)):
            if self.tasks[i][7] == 0:
                cluster = []
                cluster.append(self.tasks[i][0]) #put itself in the same cluster
                self.tasks[i][7] = 1 #change the 0 to 1, this is to check if this element is used already or not
                
                #cluster what self.tasks[i] needs in prior to be done
                if self.tasks[i][3] != []: #if it is a dependent work and is not assigned to a cluster yet
                    for k in range(len(self.tasks[i][3])):
                        cluster.append(self.tasks[i][3][k]) #then group all the dependencies
                        self.tasks[self.tasks[i][3][k]][7] = 1 #change the 0 to 1 so that it is not used again
                        #queue.append(self.tasks[self.tasks[i][3][k]][0])
                #cluster what self.tasks[i] needs to be done for the other task to be done
                queue = [self.tasks[i][0]]
                for t in self.tasks:
                    for j in queue:
                        if (j in t[3]) and (t[7] == 0):
                            cluster.append(t[0])
                            #print(t[0])
                            t[7] = 1 #change the 0 to 1 so that it is not used again
                            queue.append(t[0])
                            #queue.remove(queue[0]) #dequeue
                            queue = queue[1:] #dequeue
                            for k in t[3]:
                                if self.tasks[k-1][7] == 0: #ids = index number + 1 (in the 'tasks' input) 
                                    cluster.append(k)
                                    self.tasks[k-1][7] = 1
                                    queue.append(k)
                #print(cluster)  
                #now after the 'cluster' being made, record it to the list of clusteres = 'clustered_tasks'
                if len(cluster)>1:
                    clustered_tasks.append(cluster)
                elif (len(cluster) == 1) and (self.tasks[cluster[0]-1][7] == 0):
                    clustered_tasks.append(cluster)
            else:
                pass
            
        print(clustered_tasks)
        
        for i in clustered_tasks:
            sum_values = 0 #sum of values
            for k in i:
                sum_values += self.tasks[k-1][5] #in tasks, id starts from 1 while index of a list in python starts from 0
            for k in i:
                self.tasks[k-1][5] = sum_values #make the 'cluster' by having all elements of the cluster to have the same value of 'value' from class todo
                #this will automatically assign all the tasks that needs to be done together, go together

        return clustered_tasks  
    
    def sorting(self):
        
        #make a sorted list depending on the 'value'
        self.sequence = heap_sort(self.tasks, 5)
        
        clusts = list(chain.from_iterable(self.cluster()))
        print(clusts)
        '''
        sequence = []*len(self.tasks)
        clusts = self.cluster()
        for i in clusts:
            for j in i: #how to make it take smaller time complexity?
                if self.tasks[j-1][3] != []:
                    max_ids = max(self.tasks[j-1][3])+1 #yet, the ids = (index of self.tasks) - 1
                    #from the idea of hashing(avoiding collision), find the next place where there are no value assigned
                    if sequence[max_ids] == []:
                        sequence[max_ids] = self.tasks[j-1]
                    while sequence[max_ids] != []:
                        max_ids += 1
        '''
        for i in range(len(self.sequence)):
            if self.sequence[i][3] == []:
                pass
            else: #to avoid a loop of exchanging the two element over and over again, if statement is added
                max_ids = (clusts.index(max(self.sequence[i][3])))+1
                if self.sequence[i][0] >= max_ids:
                    pass
                else:
                    self.sequence[i], self.sequence[max_ids] = self.sequence[max_ids],self.sequence[i]

        for i in range(len(self.sequence)):
            duration_hr = self.sequence[i][2]/60
            basetime = self.hr[i][1]
            if self.sequence[i-1][4] != 0 and self.sequence[i][4]: #for the works that are together but can be done by multitasking
                basetime = self.hr[i][0] #instead of adding time on the previous work ends, start doing the work at the same time 
            self.hr.append([basetime, basetime+duration_hr, self.sequence[i]])
        print(self.hr)
        return self.hr #now we built the final list
        
    def make_list(self):
        #a method that returns self.hr with all the tasks sorted in greedy-optimal order
        self.sorting()
        return self.printout()
    
    def new_list(self,new_task): # a method for adding new task in a day
        #new_list should
        #self.tasks.appemd(new_task) -> this way I need to run the sorting again. To consume less time, let's use the already built program and insert our new idea
        ct = self.cluster()
        if int(len(self.hr))>1:
            #if the tasks are already sorted but new element comes in, insert the new task
            if len(new_task[3]) == []: #if it is not dependent
                for i in range(int(len(self.hr))):
                    if self.hr[i][1][5] > new_task[5]: #O(n) complexity, however, as there are not much tasks per day (humans don't have millions of things to do in a day)
                        #in expense of time compared to having another storage of values is fine as the time has not much difference (when the elements are in small size)
                        self.hr.append([new_task[6], new_task]) #this will change the self.hr
            else: #when the new element is a dependent one
                max_ids_dependency = ct.index(max(new_task[3])) #index of the biggest number in the dependency in the list of self.hr
                max_ids_value = 0 #finding the place according to the value from basically a same thing with self.hr
                for i in self.sequence:
                    if new_task[5] > i: #worstcase complexity of O(n)
                        max_ids_value = i
                        break
                    elif self.sequence[-1] > new_task[5]:
                        max_ids_value = len(self.sequence)-1
                    else:
                        continue
                for i in self.hr:
                    #find the max_ids and insert the new task behind it considering the importance
                    if max_ids_dependency >= max_ids_value:
                        self.hr = self.hr[:max_ids_value+1] + new_task + self.hr[max_ids_value+1:]
                    else:
                        self.hr = self.hr[:max_ids_dependency+1] + new_task + self.hr[max_ids_dependency+1:]
        return self.printout() #return the new list of to-do's for the day
        
    def printout(self):
        for i in self.hr:
            #print what we want to see
            print(str(i[0]) + 'o\'clock start, ' + str(i[1]) + 'o\'clock end : ' + i[2][1])
            

In [3]:
#ids, 'description', duration, dependencies, dependency, multi-tasking, urgency, importance
assignment = todo(1,'assignment', 600, [], 0, 4, 4)
orchestra = todo(2, 'practice for the orchestra', 60, [],0,11, 2)
read = todo(3, 'read a book: Clean Coding', 120, [],0, -1, 3)
preparation = todo(4, 'prepare for tomorrow\'s field trip lunch box', 30, [1,2], -1, 4)
review = todo(5, 'review the classes and memo the points to take with me', 90, [4], -1, 3)

#always, tasks should be in the ascending order of ids
tasks = [assignment.aslist(), orchestra.aslist() , read.aslist(), preparation.aslist(), review.aslist()]

Monday = todo_list(tasks, 6.5) #what to do on monday after waking up at 6:30 am
Monday.make_list()

pt = todo(6, 'go to work(part time job) to pay for the tuition', 240, [1], True)
parttime = pt.aslist()
Monday.new_list(parttime)

[[5, 4], [3], [4, 1, 2]]
[5, 4, 3, 4, 1, 2]


IndexError: list index out of range

In [6]:
[1] + [2]


[1, 2]